In [ ]:
import pandas as pd
import os
import requests
import json
import csv
import io
from collections import defaultdict
from time import time
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
#from flair.data import Sentence
#from flair.nn import Classifier
from chembl_webresource_client.new_client import new_client

# 1. Get targets from disease

In [ ]:
"""
insert code here that
1. Takes an EFO as input
2. displays a dataframe of targets for that disease
3. uploads that dataframe to a table in planetscale called "disease_to_target"

"""
from sqlalchemy import create_engine
from dotenv import dotenv_values
config = dotenv_values('database_url.env')
url = config['DATABASE_URL']

engine = create_engine(url, echo=False)



## Set disease_id variable for desired disease

disease_id = "EFO_0005537"

disease_df = pd.read_sql("SELECT * FROM disease_to_target WHERE disease_id = '{disease_id}';".format(disease_id = disease_id), con=engine)
display(disease_df)




,level_0,index,disease_id,target_ensemble_id,target_chembl_id,association_score
0,0,0,EFO_0005537,ENSG00000184292,CHEMBL3856163,0.431490
1,1,1,EFO_0005537,ENSG00000198900,CHEMBL1781,0.424207
2,2,2,EFO_0005537,ENSG00000120217,CHEMBL3580522,0.405569
3,3,3,EFO_0005537,ENSG00000258947,CHEMBL2597,0.402827
4,4,4,EFO_0005537,ENSG00000137267,CHEMBL3797012,0.399973
...,...,...,...,...,...,...
5012,5012,5012,EFO_0005537,ENSG00000135164,CHEMBL3707469,0.001478
5013,5013,5013,EFO_0005537,ENSG00000115665,CHEMBL4507,0.001478
5014,5014,5014,EFO_0005537,ENSG00000146109,CHEMBL4507,0.001478
5015,5015,5015,EFO_0005537,ENSG00000225830,CHEMBL4507,0.001478


# 2. Get compounds and assays

In [ ]:
"""
insert code here that
1. takes a target as input
2. searches the database for compounds and assays for that target
3. displays a dataframe with at least these columns: target id, compound id, assay id
"""
engine = create_engine(url, echo=False)
target_id = 'ENSG00000120217'

target_to_compounds_df = pd.read_sql("SELECT * FROM target_to_compounds WHERE target_ensemble_id='{target}';".format(target=target_id), con=engine)
display(target_to_compounds_df)





,level_0,index,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_id,abstract,target_ensemble_id
0,1895,0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
1,1896,1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269,B,Binding affinity to PDL1 (unknown origin) asse...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
2,1897,2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
3,1898,3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
4,1899,4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B,Inhibition of PD-L1 (unknown origin) by HTRF a...,CHEMBL5096155,Tumor immunotherapy has made great progress in...,ENSG00000120217
5,1900,5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217
6,1901,6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217
7,1902,7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B,Inhibition of PD-L1 (unknown origin),CHEMBL4261578,The modulation of immune checkpoint receptors ...,ENSG00000120217
8,1903,8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL3580542,,ENSG00000120217
9,1904,9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217


# 3. Propose a new compound

In [ ]:
"""
insert code here that
1. trains a model to predict ic50 values for a compound on a given target
2. downloads ALL the compounds in the database (independent of target)
3. runs the model to predict ic50 values for each compound in the database
4. displays a dataframe that has the following two columns : compound, predicted ic50
5. prints out the compound that has the highest ic50 value that has NOT been tested on the target already.
"""
# 1. Train Model
import deepchem as dc
import numpy as np

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

#!git clone https://github.com/dsalinasduron-msmary/chemical_informatics.git

import os
import pickle
#os.listdir()
#os.chdir('chemical_informatics')
#os.getcwd()

f = open('ENSG00000128191dataframe.pickle','rb')
compound_dataset = pickle.load(f)
f.close()
display(compound_dataset)

smiles = compound_dataset['smiles']
IC50 = compound_dataset['standard value']
featurizer = dc.feat.ConvMolFeaturizer()
compound_dataset['featurized'] = featurizer.featurize(smiles)
compound_dataset['divided values'] = compound_dataset['standard value'].astype(float).div(108000)
compound_dataset['pIC50'] = np.log10(compound_dataset['divided values'].astype(float)).mul(-1)
compound_dataset['number'] = list(range(0,len(compound_dataset)))
display(compound_dataset)

#compound_dataset['pIC50'].hist()

x = compound_dataset
f = open('x.pickle','wb')
pickle.dump(x,f)
f.close()

import pandas as pd
training_dataset = compound_dataset.sample(frac = 0.7)

#training_dataset.featurized[0].n_feat

testing_dataset = (compound_dataset[~compound_dataset['number'].isin(training_dataset['number'])])
display(testing_dataset)

numpy_training_dataset = dc.data.NumpyDataset(X=training_dataset['featurized'],y=training_dataset['pIC50'].astype(float), ids=training_dataset['smiles'])
numpy_testing_dataset = dc.data.NumpyDataset(X=testing_dataset['featurized'],y=testing_dataset['pIC50'].astype(float), ids=testing_dataset['smiles'])
display(numpy_training_dataset)
display(numpy_testing_dataset)

model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10, model_dir="/content/chemical_informatics")

model.fit(numpy_training_dataset, nb_epoch=5000)

model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10, model_dir="/content/chemical_informatics")
model.restore()

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(numpy_training_dataset, [metric]))
print("Test set score:", model.evaluate(numpy_testing_dataset, [metric]))


#2. Download all compounds in database
with engine.begin() as conn:
    query = text("SELECT smiles FROM target_to_compounds;")
    compounds_df = pd.read_sql(query, conn)

predict_list = []
for smile in range(0, len(compounds_df)):
    predict_list.append(None)
print(len(predict_list))

compounds_df['predicted_pIC50'] = predict_list

curated_compounds_df = compounds_df.dropna(subset=['smiles'])

#3. run the model to predict ic50 values for each compound in the database
dataset = dc.data.NumpyDataset(X=curated_compounds_df['featurized'], y=curated_compounds_df['predicted_pIC50'], ids=curated_compounds_df['smiles'])
curated_compounds_df['predicted_pIC50'] = model.predict(dataset)

# 4. displays a dataframe that has the following two columns : compound, predicted ic50
display(curated_compounds_df)




# 4. Propose an assay

In [ ]:
#!pip install seaborn

In [ ]:
import seaborn
from matplotlib.pyplot import hist
from sklearn.neighbors import KernelDensity
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

In [ ]:
"""
insert code here that
1. retrieves the terms for all the assays that are relevant to the target the user picked.
2. clusters the assays according to their descriptive terms
3. plots the clusters (set n_clusters = 10)
4. prints out the title of one assay from each cluster.
"""

config = dotenv_values("database_URL.env")
url = config['DATABASE_URL']
engine = create_engine(url, echo=False)

with engine.begin() as conn:
    query = text("select * from target_to_compounds;")
    target_list = pd.read_sql(query, conn)
display(target_list)


three_col_list = target_list[['assay_id', 'assay_description', 'abstract']].copy()

unique_list = []
for index, row in three_col_list.iterrows():
    if (row["assay_id"], row["assay_description"], row["abstract"]) in unique_list:
        continue
    else:
        unique_list.append((row["assay_id"], row["assay_description"], row["abstract"]))

Assay_Descriptions = [abstract for (assay_ids, assay_name, abstract) in unique_list]
Assay_Descriptions_Joined = ':: '.join(Assay_Descriptions)

print(len(unique_list))
#print(unique_list)
#print(Assay_Descriptions)


tagger = Classifier.load('hunflair')
print("Finished setting tagger")

Assay_Descriptions_List = []

sentence = Sentence(Assay_Descriptions_Joined)
tagger.predict(sentence)

Assay_Key_Words = ""
for label in sentence.get_labels():
    Assay_Key_Words += label.data_point.text
    Assay_Key_Words += " "
    #print(label.data_point.text)
    #print(Assay_Key_Words)

#adds the Descriptions of the Assays to the Assay Description List
Assay_Descriptions_List.append(Assay_Key_Words)

def tokenize_with_flair(description):
    sentence = Sentence(description)
    tagger.predict(sentence)
    return [ l.data_point.text for l in sentence.get_labels() ]



Assay_Count_Vect = CountVectorizer(tokenizer=tokenize_with_flair)
Assay_Train_Counts = Assay_Count_Vect.fit_transform(Assay_Descriptions_List)

n_clusters = 10  # number of regions
X = Assay_Train_Counts.toarray()

ward = AgglomerativeClustering(
    n_clusters = n_clusters, linkage="ward", connectivity=None, compute_full_tree= True,compute_distances= True
)
ward.fit(X)

print(ward.distances_)
print(ward.distances_.shape)

X_Hist = ward.distances_
#hist(X_Hist)
kernel = stats.gaussian_kde(X_Hist)
print(kernel(X_Hist))

#first dendogram
seaborn.clustermap(X,method='ward')

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(Assay_Train_Counts)


# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_,index=Assay_Count_Vect.get_feature_names(),columns=["idf_weights"])

# sort ascending
df_idf.sort_values(by=['idf_weights'])

# count matrix
count_vector=Assay_Count_Vect.transform(Assay_Descriptions_List)

# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
print(tf_idf_vector)
tf_idf_vector.shape

feature_names = Assay_Count_Vect.get_feature_names()

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]

#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


#second dendogram
seaborn.clustermap(tf_idf_vector.toarray(),method='ward')

Finished Installing flair
200
done
ENSG00000184292
0
0
ENSG00000198900
456
456
ENSG00000120217
34
490


,target_ENSG,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_ID,abstract
0,ENSG00000198900,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
1,ENSG00000198900,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
2,ENSG00000198900,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
3,ENSG00000198900,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
4,ENSG00000198900,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
...,...,...,...,...,...,...,...,...,...,...
485,ENSG00000120217,CHEMBL4791550,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[...,IC50,5600.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
486,ENSG00000120217,CHEMBL4798338,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,19200.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
487,ENSG00000120217,CHEMBL366760,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H]1C[C@@](O)(C(=O)...,IC50,36560.0,CHEMBL4613347,B,Antagonist activity against PDL1 (unknown origin),CHEMBL4613205,Blockade the interaction of the programmed cel...
488,ENSG00000120217,CHEMBL4790749,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,80700.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...


Finished setting tagger
